In [2]:
#model with CRF Max_len = 75


import pandas as pd
import numpy as np
from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras_contrib.layers import CRF
import numpy
numpy.random.seed(7)
data = pd.read_csv('/home/sharath/Documents/torchdata/lmscsv/bcmtokens.csv', encoding='latin1', sep = ',')
data = data.fillna(method="ffill")
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences
max_len = 50
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}


import numpy as np

# Save
np.save('/home/sharath/Documents/savedweights_Bilstm/word2idxcrfbcm.npy', word2idx) 

# Load
#word2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\word2idx.npy').item()


# Save
np.save('/home/sharath/Documents/savedweights_Bilstm/tag2idxcrfbcm.npy', tag2idx) 

# Load
#tag2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\tag2idx.npy').item()


X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

model = Model(input, out)

model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
history = model.fit(X_tr, np.array(y_tr), batch_size=20, epochs=150,
                    validation_split=0.1, verbose=1)

Train on 48 samples, validate on 6 samples
Epoch 1/150
48/48 [==============================] - 3s 55ms/step - loss: 7.4105 - acc: 0.0011 - val_loss: 7.1299 - val_acc: 0.0000e+00
Epoch 2/150
48/48 [==============================] - 0s 6ms/step - loss: 7.3218 - acc: 0.0340 - val_loss: 7.0071 - val_acc: 0.2549
Epoch 3/150
48/48 [==============================] - 0s 3ms/step - loss: 7.1331 - acc: 0.2943 - val_loss: 6.7136 - val_acc: 0.4575
Epoch 4/150
48/48 [==============================] - 0s 4ms/step - loss: 6.7545 - acc: 0.5254 - val_loss: 6.9553 - val_acc: 0.4641
Epoch 5/150
48/48 [==============================] - 0s 3ms/step - loss: 6.6395 - acc: 0.5234 - val_loss: 6.7120 - val_acc: 0.4641
Epoch 6/150
48/48 [==============================] - 0s 3ms/step - loss: 6.5969 - acc: 0.5246 - val_loss: 6.7089 - val_acc: 0.4641
Epoch 7/150
48/48 [==============================] - 0s 3ms/step - loss: 6.5298 - acc: 0.5254 - val_loss: 6.6999 - val_acc: 0.4641
Epoch 8/150
48/48 [================

Epoch 63/150
48/48 [==============================] - 0s 4ms/step - loss: 5.2309 - acc: 0.8456 - val_loss: 5.4167 - val_acc: 0.7647
Epoch 64/150
48/48 [==============================] - 0s 4ms/step - loss: 5.2076 - acc: 0.8304 - val_loss: 5.3535 - val_acc: 0.7712
Epoch 65/150
48/48 [==============================] - 0s 4ms/step - loss: 5.1758 - acc: 0.8630 - val_loss: 5.3168 - val_acc: 0.8170
Epoch 66/150
48/48 [==============================] - 0s 4ms/step - loss: 5.1666 - acc: 0.8740 - val_loss: 5.3897 - val_acc: 0.7647
Epoch 67/150
48/48 [==============================] - 0s 5ms/step - loss: 5.1507 - acc: 0.8614 - val_loss: 5.3305 - val_acc: 0.7843
Epoch 68/150
48/48 [==============================] - 0s 4ms/step - loss: 5.1289 - acc: 0.8818 - val_loss: 5.3190 - val_acc: 0.7843
Epoch 69/150
48/48 [==============================] - 0s 5ms/step - loss: 5.1160 - acc: 0.8791 - val_loss: 5.2794 - val_acc: 0.7908
Epoch 70/150
48/48 [==============================] - 0s 4ms/step - loss: 5.

Epoch 125/150
48/48 [==============================] - 0s 5ms/step - loss: 4.7255 - acc: 0.9894 - val_loss: 5.6103 - val_acc: 0.8170
Epoch 126/150
48/48 [==============================] - 0s 3ms/step - loss: 4.7266 - acc: 0.9897 - val_loss: 5.4285 - val_acc: 0.8627
Epoch 127/150
48/48 [==============================] - 0s 4ms/step - loss: 4.7180 - acc: 0.9948 - val_loss: 5.5224 - val_acc: 0.8301
Epoch 128/150
48/48 [==============================] - 0s 4ms/step - loss: 4.7212 - acc: 0.9905 - val_loss: 5.4628 - val_acc: 0.8431
Epoch 129/150
48/48 [==============================] - 0s 3ms/step - loss: 4.7191 - acc: 0.9876 - val_loss: 5.4205 - val_acc: 0.8627
Epoch 130/150
48/48 [==============================] - 0s 4ms/step - loss: 4.7177 - acc: 0.9884 - val_loss: 5.5616 - val_acc: 0.8366
Epoch 131/150
48/48 [==============================] - 0s 4ms/step - loss: 4.7172 - acc: 0.9884 - val_loss: 5.6108 - val_acc: 0.8170
Epoch 132/150
48/48 [==============================] - 0s 3ms/step - 

In [9]:
test_sentence = "The architecture of Hatch washer consists of a gateway connection to Controller Area Network signal BUS and a combination BUS.".split(" ")

In [10]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [11]:
def newpredict(tt):
    result = []
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in tt]],
                            padding="post", value=0, maxlen=max_len)
    p = model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(tt, p[0]):
        result.append((w, tags[pred]))
    return result

In [12]:
newpredict(test_sentence)

[('The', 'O'),
 ('architecture', 'O'),
 ('of', 'O'),
 ('Hatch', 'auxillary system'),
 ('washer', 'auxillary system'),
 ('consists', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('gateway', 'system component'),
 ('connection', 'system component'),
 ('to', 'O'),
 ('Controller', 'system component'),
 ('Area', 'system component'),
 ('Network', 'system component'),
 ('signal', 'system_signal'),
 ('BUS', 'system_signal'),
 ('and', 'O'),
 ('a', 'O'),
 ('combination', 'O'),
 ('BUS.', 'system component')]

In [13]:
[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

In [14]:
scores = model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

54/54 [==============================] - 0s 2ms/step
acc: 98.36%


In [15]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
custom_objects={'CRF':CRF}

In [17]:
from keras.models import model_from_json
from keras_contrib.layers import CRF
# load json and create model
json_file = open('/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json, custom_objects={'CRF':CRF})
# load weights into new model
loaded_model.load_weights("/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.h5")
print("Loaded model from disk")

Loaded model from disk


In [28]:
import os
os.getcwd()

'/home/sharath'

In [117]:
#loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [121]:
# score = loaded_model.evaluate(X_tr, np.array(y_tr), verbose=1)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [21]:
def bilstmfinalcrf(test_sentence):
    from keras.models import load_model
    from keras_contrib.layers import CRF, crf
    import pandas as pd
    import numpy as np
    from keras.models import model_from_json
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Model, Input
    from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import numpy
    numpy.random.seed(7)
    word2idx = np.load('/home/sharath/Documents/savedweights_Bilstm/word2idxcrfbcm.npy').item()
    tag2idx = np.load('/home/sharath/Documents/savedweights_Bilstm/tag2idxcrfbcm.npy').item()
    # load json and create model
    json_file = open('/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json,custom_objects={'CRF':CRF})
    # load weights into new model
    loaded_model.load_weights("/home/sharath/Documents/savedweights_Bilstm/modelcrfbcm.h5")
    loaded_model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
    test_sentence = test_sentence.split(" ")
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=50)
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    pls = []
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(test_sentence, p[0]):
        pls.append(pred)
    revsubs = { v:k for k,v in tag2idx.items()}
    entit = [revsubs.get(item,item) for item in pls]
    final = list(zip(test_sentence,entit))
    return final

In [22]:
s = "The architecture of Hatch washer consists of a gateway connection to Controller Area Network signal BUS and a combination BUS."

In [23]:
bilstmfinalcrf(s)

AttributeError: module 'keras_contrib.layers.crf' has no attribute 'loss_function'

In [ ]:
[('Half', 'Main_Application'),
 ('day', 'O'),
 ('leave', 'O'),
 ('of', 'O'),
 ('CL', 'Leave_category'),
 ('(', 'O'),
 ('Casual', 'Leave_category'),
 ('Leave', 'App_feature'),
 (')', 'O'),
 ('is', 'O'),
 ('allowed', 'O'),
 ('for', 'O'),
 ('Mechanic', 'Title'),
 ('employees.', 'App_feature')]

In [84]:
# def lstm_model():
#     import pandas as pd
#     import numpy as np
#     from keras.models import load_model


#     data = pd.read_csv('C:\\users\\sharath\\postags_v1.csv', encoding='latin1', sep = ',')
#     data = data.fillna(method="ffill")
#     words = list(set(data["Word"].values))
#     words.append("ENDPAD")
#     n_words = len(words)
#     tags = list(set(data["Tag"].values))
#     n_tags = len(tags)

#     class SentenceGetter(object):

#         def __init__(self, data):
#             self.n_sent = 1
#             self.data = data
#             self.empty = False
#             agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
#                                                                s["POS"].values.tolist(),
#                                                                s["Tag"].values.tolist())]
#             self.grouped = self.data.groupby("Sentence #").apply(agg_func)
#             self.sentences = [s for s in self.grouped]

#         def get_next(self):
#             try:
#                 s = self.grouped["Sentence: {}".format(self.n_sent)]
#                 self.n_sent += 1
#                 return s
#             except:
#                 return None

#     getter = SentenceGetter(data)
#     sent = getter.get_next()
#     sentences = getter.sentences
#     max_len = 50
#     word2idx = {w: i for i, w in enumerate(words)}
#     tag2idx = {t: i for i, t in enumerate(tags)}
#     from keras.preprocessing.sequence import pad_sequences
#     X = [[word2idx[w[0]] for w in s] for s in sentences]
#     X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
#     y = [[tag2idx[w[2]] for w in s] for s in sentences]
#     y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#     from keras.utils import to_categorical
#     y = [to_categorical(i, num_classes=n_tags) for i in y]
#     from sklearn.model_selection import train_test_split
#     X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
#     from keras.models import Model, Input
#     from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
#     input = Input(shape=(max_len,))
#     model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
#     model = Dropout(0.1)(model)
#     model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
#     out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
#     model = Model(input, out)
#     model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
#     history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=10, validation_split=0.1, verbose=1)
#     history.model.save('m3odel.h5')  # creates a HDF5 file 'my_model.h5'
#     del model  # deletes the existing model
#     # returns a compiled model
#     # identical to the previous one
#     modelkk = load_model('m3odel.h5')
#     return modelkk

In [2]:
import tensorflow as tf

In [5]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'MatMul_1': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a, b)]]

Caused by op 'MatMul_1', defined at:
  File "C:\Users\sharath\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sharath\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sharath\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sharath\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sharath\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance

In [4]:
sess

In [8]:
tt = "The system shall allow the data manager to search occurrence resources by keywords"

In [9]:
tt = "The system shall allow the data manager to edit the metadata of an institution resource"